# Calculation of validation sample size using a combined approach of equal and proportional sample distribution

### Using the results from CART, RF and SVM classification
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2022

Author: Finn Geiger\
Date: April 5th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [2]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [3]:
#ee.Authenticate() 
#ee.Initialize()

In [4]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [5]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2022_12 = nicfi.filter(ee.Filter.date('2022-12-01','2023-01-01')).first()

# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2022_12, 4)
Map.addLayer(basemap_2022_12, vis_params, '2022-12 mosaic')

In [6]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [7]:
# clipping the basescene to the AOI
basescene = basemap_2022_12.clipToCollection(aoi_windhoek)
Map.addLayer(basescene, vis_params, 'clipped')

#### 1.3 importing the classification results and adding them to the map

In [8]:
# importing the classified maps and  adding them to the map
classified_cart = ee.Image('users/s85315/masterthesis/classification_results/classified_basescene_CART')
classified_rf = ee.Image('users/s85315/masterthesis/classification_results/classified_basescene_RF')
classified_svm = ee.Image('users/s85315/masterthesis/classification_results/classified_basescene_SVM')

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


Map.addLayer(classified_cart, vis_params_classified, 'classified basescene CART')
Map.addLayer(classified_rf, vis_params_classified, 'classified basescene RF')
Map.addLayer(classified_svm, vis_params_classified, 'classified basescene SVM')

### 2 Calculation of the class sizes from pixel size

#### 2.1 Calculation for CART classification map

In [34]:
# calculating pixel size
area_classified_cart = ee.Image.pixelArea().addBands(classified_cart)

# calculating the area size per class
areas = area_classified_cart.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'landcover',
    }),
    'geometry': aoi_windhoek, 
    'scale': 4.77
    })

# selecting the size per class
class_areas = ee.List(areas.get('groups'))
list_dict = class_areas.getInfo()

# creating a dataframe for easier data handling
df_areas_cart = pd.DataFrame.from_records(list_dict)
df_areas_cart['sum'] = df_areas_cart['sum'].astype(int)
df_areas_cart.to_csv('./data/class_size_cart.csv', sep=";", index=False)

print(tabulate(df_areas_cart, tablefmt='fancy_grid', headers=['Landcover', 'Area size [m²]'], showindex=False, floatfmt=".0f"))

╒═════════════╤══════════════════╕
│   Landcover │   Area size [m²] │
╞═════════════╪══════════════════╡
│           1 │          6502663 │
├─────────────┼──────────────────┤
│           2 │          4163721 │
├─────────────┼──────────────────┤
│           3 │          2258723 │
├─────────────┼──────────────────┤
│           4 │         10464735 │
├─────────────┼──────────────────┤
│           5 │         52655990 │
├─────────────┼──────────────────┤
│           6 │          4685100 │
├─────────────┼──────────────────┤
│           7 │           951473 │
╘═════════════╧══════════════════╛


##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilecart